In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


*****Loading data*****

In [ ]:
data_path = "/kaggle/input/lish-moa/"
train_features = pd.read_csv(data_path + 'train_features.csv')
scored = pd.read_csv(data_path + 'train_targets_scored.csv')
non_scored=pd.read_csv(data_path + 'train_targets_nonscored.csv')
drug=pd.read_csv(data_path + 'train_drug.csv')
test = pd.read_csv(data_path + 'test_features.csv')
sample_submission = pd.read_csv(data_path + 'sample_submission.csv')


**show the data & data information**

In [ ]:
print("Train Data: ")
print("Shape:"+str(train_features.shape))
train_features.head(5)

In [ ]:
train_features.info()

In [ ]:
print("Targets Scored: ")
print("Shape:"+str(scored.shape))
scored.head(5)



In [ ]:
scored.info()

In [ ]:
print("Targets NonScored: ")
print("Shape:"+str(non_scored.shape))
non_scored.head(5)

In [ ]:
non_scored.info()

In [ ]:
print("Drug: ")
print("Shape:"+str(drug.shape))
drug.head(5)

In [ ]:
drug.info()

**Check Missing Values**

In [ ]:
print("Training missing data:", train_features.isnull().sum().sum())
print("Testing missing data:", test.isnull().sum().sum())
print("Target missing data:", scored.isnull().sum().sum())

**check for duplicates**

In [ ]:

print("Training duplicated data:", train_features.duplicated().sum())
print("Testing duplicated data:", test.duplicated().sum())
print("Target duplicated data:", scored.duplicated().sum())

**Show theGene expression &cell viability coloumns number**

In [ ]:
train_gs = train_features.iloc[:,train_features.columns.map(lambda x: x[0:2])=='g-']
train_cs = train_features.iloc[:,train_features.columns.map(lambda x: x[0:2])=='c-']
print("Gene expression data Number of columns: "+str(train_gs.shape[1]))
print("cell viability data Number of columns: "+ str(train_cs.shape[1]))

In [ ]:
train_features.describe()

In [ ]:
print("Mean:"+str(pd.concat([train_gs,train_cs],axis=1).values.mean()))
print("Std:"+str(pd.concat([train_gs,train_cs],axis=1).values.std()))
plt.figure(figsize=(5,5))
sns.distplot(pd.concat([train_gs,train_cs],axis=1).values)
plt.title('combined gene expression and cell viability')
plt.figure(figsize=(12,12))
plt.subplot(2,2,1)
sns.distplot(train_gs['g-0'],color='red')
plt.title('g-0')
plt.subplot(2,2,2)
sns.distplot(train_gs['g-100'],color='red')
plt.title('g-100')
plt.subplot(2,2,3)
sns.distplot(train_cs['c-1'],color='red')
plt.title('c-0')
plt.subplot(2,2,4)
sns.distplot(train_cs['c-80'],color='red')
plt.title('c-80')

**Gene expression & Cell viability statistics**

In [ ]:
print("Gene expression data statistics: ")
print("  Mean: "+str(train_gs.values.mean()))
print("  Std: "+str(train_gs.values.std()))
print("  Max: "+str(train_gs.values.max()))
print("  Min: "+str(train_gs.values.min()))
print('\nCell viability data statistics: ')
print("  Mean: "+str(train_cs.values.mean()))
print("  Std: "+str(train_cs.values.std()))
print("  Max: "+str(train_cs.values.max()))
print("  Min: "+str(train_cs.values.min()))

In [ ]:
plt.figure(figsize=(6,6))
ax = sns.countplot(train_features["cp_dose"] , palette="Set2")
ax.set_title("Treatment Dose")


plt.show()



In [ ]:
plt.figure(figsize=(6,6))
ax = sns.countplot(train_features["cp_time"] , palette="Set2")
ax.set_title("Treatment time")


plt.show()

In [ ]:
plt.figure(figsize=(6,6))
ax = sns.countplot(train_features["cp_type"] , palette="Set2")
ax.set_title("Treatment Type")


In [ ]:
plt.figure(figsize=(8,15))
plt.subplot(3,1,1)
splot = sns.countplot(train_features["cp_type"])
for p in splot.patches:
    splot.annotate(format(p.get_height(), '.1f'), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', 
                   xytext = (0, 9), 
                   textcoords = 'offset points')
plt.title('cp_type')
plt.subplot(3,1,2)
sns.countplot(train_features['cp_time'],hue=train_features['cp_type'])
plt.title('cp_time vs cp_type')
plt.subplot(3,1,3)
sns.countplot(train_features['cp_dose'],hue=train_features['cp_type'])
plt.title('cp_dose vs cp_type')
plt.tight_layout()

In [ ]:
print("Number of scored targets: "+str(scored.shape[1]))

**Exploring some relationships**

we can use stripplot to show the relationship of a feature and a target with respect to dosage and time.

In [ ]:
out = dict()
arr=scored.drop('sig_id',axis=1).values==1
for a in range(len(arr)):
    o=np.sum(arr[a])
    if o not in out.keys():
        out[o]=1
    else:
        out[o]+=1
length = 23814
plt.figure(figsize=(7,7))
splot = sns.barplot(x=list(out.keys()),y=list(out.values()))
for p in splot.patches:
    splot.annotate(format(p.get_height()*100/length, '.1f')+'%', 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', 
                   xytext = (0, 9), 
                   textcoords = 'offset points')
plt.xlabel('Number of MoAs in sample')
plt.ylabel('Count')
plt.title('Percentage of samples with MoA counts')

In [ ]:
cor = scored.drop('sig_id',axis=1).corr()

In [ ]:
cor = scored.drop('sig_id',axis=1).corr()
plt.figure(figsize=(10,10))
sns.heatmap(cor)

In [ ]:
df = pd.DataFrame(columns=['drug_a','drug_b','corr'])
for j in range(len(cor)):
    for i in range(len(cor)):
        if cor.iloc[i,j]>=0.7 and cor.iloc[i,j]!=1.0:
            df = pd.concat([df,pd.DataFrame({'drug_a':[cor.columns[j]],'drug_b':[cor.columns[i]],'corr':[cor.iloc[i,j]]})],axis=0)
df

In [ ]:
print("Number of non scored targets: "+str(non_scored.shape[1]))

In [ ]:
out = dict()
arr=pd.concat([scored.drop('sig_id',axis=1),non_scored.drop('sig_id',axis=1)],axis=1).values==1
for a in range(len(arr)):
    o=np.sum(arr[a])
    if o not in out.keys():
        out[o]=1
    else:
        out[o]+=1
length = 23814
plt.figure(figsize=(7,7))
splot = sns.barplot(x=list(out.keys()),y=list(out.values()))
for p in splot.patches:
    splot.annotate(format(p.get_height()*100/length, '.1f')+'%', 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', 
                   xytext = (0, 9), 
                   textcoords = 'offset points')
plt.xlabel('Number of MoAs in sample')
plt.ylabel('Count')
plt.title('Percentage of samples with MoA counts')

In [ ]:
genes = [col for col in train_features if col.startswith('g-')]
cells = [col for col in train_features if col.startswith('c-')]

In [ ]:
plt.figure(figsize=(8,8))
sns.heatmap(train_features.loc[:,genes].corr(),cmap='viridis')

In [ ]:
plt.figure(figsize=(8,8))
sns.heatmap(train_features.loc[:,cells].corr(),cmap='viridis')

In [ ]:
cor = train_gs.corr()

In [ ]:
df = pd.DataFrame(columns=['gene_a','gene_b','corr'])
for j in range(len(cor)):
    for i in range(len(cor)):
        if cor.iloc[i,j]<=-0.8 and cor.iloc[i,j]!=1.0:
            df = pd.concat([df,pd.DataFrame({'gene_a':[cor.columns[j]],'gene_b':[cor.columns[i]],'corr':[cor.iloc[i,j]]})],axis=0)
df

In [ ]:
cp_1 = train_features[train_features['cp_type']=='trt_cp']
cp_2 = train_features[train_features['cp_type']!='trt_cp']
plt.figure(figsize=(10,10))
plt.subplot(2,2,1)
sns.distplot(cp_1['g-0'],color='orange',hist=False)
sns.distplot(cp_2['g-0'],color='blue',hist=False)
plt.title('g-0')
plt.subplot(2,2,2)
sns.distplot(cp_1['g-100'],color='orange',hist=False)
sns.distplot(cp_2['g-100'],color='blue',hist=False)
plt.title('g-100')
plt.subplot(2,2,3)
sns.distplot(cp_1['g-500'],color='orange',hist=False)
sns.distplot(cp_2['g-500'],color='blue',hist=False)
plt.title('g-500')
plt.subplot(2,2,4)
sns.distplot(cp_1['g-600'],color='orange',hist=False)
sns.distplot(cp_2['g-600'],color='blue',hist=False)
plt.title('g-600')

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(2,2,1)
sns.distplot(cp_1['c-1'],color='orange',hist=False)
sns.distplot(cp_2['c-1'],color='blue',hist=False)
plt.title('c-1')
plt.subplot(2,2,2)
sns.distplot(cp_1['c-20'],color='orange',hist=False)
sns.distplot(cp_2['c-20'],color='blue',hist=False)
plt.title('c-20')
plt.subplot(2,2,3)
sns.distplot(cp_1['c-40'],color='orange',hist=False)
sns.distplot(cp_2['c-40'],color='blue',hist=False)
plt.title('c-40')
plt.subplot(2,2,4)
sns.distplot(cp_1['c-50'],color='orange',hist=False)
sns.distplot(cp_2['c-50'],color='blue',hist=False)
plt.title('c-50')

In [ ]:
cp_1 = train_features[train_features['cp_time']==24]
cp_2 = train_features[train_features['cp_type']!=48]
cp_3 = train_features[train_features['cp_type']!=72]
plt.figure(figsize=(10,10))
plt.subplot(2,2,1)
sns.distplot(cp_1['g-0'],color='orange',hist=False)
sns.distplot(cp_2['g-0'],color='green',hist=False)
sns.distplot(cp_3['g-0'],color='blue',hist=False)
plt.title('g-0')
plt.subplot(2,2,2)
sns.distplot(cp_1['g-100'],color='orange',hist=False)
sns.distplot(cp_2['g-100'],color='green',hist=False)
sns.distplot(cp_3['g-100'],color='blue',hist=False)
plt.title('g-100')
plt.subplot(2,2,3)
sns.distplot(cp_1['g-500'],color='orange',hist=False)
sns.distplot(cp_2['g-500'],color='green',hist=False)
sns.distplot(cp_3['g-500'],color='blue',hist=False)
plt.title('g-500')
plt.subplot(2,2,4)
sns.distplot(cp_1['g-600'],color='orange',hist=False)
sns.distplot(cp_2['g-600'],color='green',hist=False)
sns.distplot(cp_3['g-600'],color='blue',hist=False)
plt.title('g-600')

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(2,2,1)
sns.distplot(cp_1['c-1'],color='orange',hist=False)
sns.distplot(cp_2['c-1'],color='green',hist=False)
sns.distplot(cp_3['c-1'],color='blue',hist=False)
plt.title('c-1')
plt.subplot(2,2,2)
sns.distplot(cp_1['c-20'],color='orange',hist=False)
sns.distplot(cp_2['c-20'],color='green',hist=False)
sns.distplot(cp_3['c-20'],color='blue',hist=False)
plt.title('c-20')
plt.subplot(2,2,3)
sns.distplot(cp_1['c-40'],color='orange',hist=False)
sns.distplot(cp_2['c-40'],color='green',hist=False)
sns.distplot(cp_3['c-40'],color='blue',hist=False)
plt.title('c-40')
plt.subplot(2,2,4)
sns.distplot(cp_1['c-50'],color='orange',hist=False)
sns.distplot(cp_2['c-50'],color='green',hist=False)
sns.distplot(cp_3['c-50'],color='blue',hist=False)
plt.title('c-50')

In [ ]:
cp_1 = train_features[train_features['cp_dose']=='D0']
cp_2 = train_features[train_features['cp_dose']!='D1']
plt.figure(figsize=(10,10))
plt.subplot(2,2,1)
sns.distplot(cp_1['g-0'],color='red',hist=False)
sns.distplot(cp_2['g-0'],color='green',hist=False)
plt.title('g-0')
plt.subplot(2,2,2)
sns.distplot(cp_1['g-100'],color='red',hist=False)
sns.distplot(cp_2['g-100'],color='green',hist=False)
plt.title('g-100')
plt.subplot(2,2,3)
sns.distplot(cp_1['g-500'],color='red',hist=False)
sns.distplot(cp_2['g-500'],color='green',hist=False)
plt.title('g-500')
plt.subplot(2,2,4)
sns.distplot(cp_1['g-600'],color='red',hist=False)
sns.distplot(cp_2['g-600'],color='green',hist=False)
plt.title('g-600')